In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict

from git import Repo
import os

In [2]:
meta = pd.read_json('D:/programs/projects/yandex_music/meta.json', lines= True, convert_dates=['dttm'])
meta.info()
display(meta.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   track_id  71768 non-null  object        
 1   dttm      71768 non-null  datetime64[ns]
 2   title     71768 non-null  object        
 3   language  21969 non-null  object        
 4   isrc      71455 non-null  object        
 5   genres    71768 non-null  object        
 6   duration  71768 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 3.8+ MB


,track_id,dttm,title,language,isrc,genres,duration
49558,91b4904cbd270d1d7a5ce5aa08c84996,2013-03-11 20:00:00,Breakaway,None,USJCH1200323,[POP],238760.0
3520,74836d21f84f038523d0a3955cdf5a50,2022-01-14 18:26:35,Scum - Dethroned Emperor,None,US7VG2259991,"[CLASSICMETAL, METAL]",255660.0
28745,c9a350d03577f7d5f025d4c1ccf452f9,2021-07-27 19:32:26,Sympathy for the Devil,EN,QZGWX2217149,"[SOUL, RNB]",391940.0
7799,5af18fdffaacb27409936a952f3776d6,2022-03-16 21:44:59,Lista de Schindler,None,QZHZ32217643,[FOLK],213140.0
57113,c607ccd64a72d7bf6b1c2f7e03086e50,2020-10-23 02:25:36,Cuando Calienta El Sol / Ahora Te Puedes March...,None,QZNJW2000863,[PUNK],248540.0


In [3]:
#load covers
covers = pd.read_json('D:/programs/projects/yandex_music/covers.json', lines= True)
covers.info()
display(covers.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71597 entries, 0 to 71596
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_track_id  4821 non-null   object
 1   track_id           71597 non-null  object
 2   track_remake_type  71597 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


,original_track_id,track_id,track_remake_type
51174,None,4e77c0453e3e68faada590c9e3d60aab,COVER
53375,None,349f8b41dd1f5751541aa5725fd6ebec,COVER
32292,None,ba7d21a28441208aa8679e0d5480d43b,COVER
5755,None,9fcde813fd39868f93071f340ea0d090,COVER
66935,None,22c8311fbf444e5fd82a3f7bd26576df,COVER


# Работа с meta

In [4]:
meta['isrc'] = meta['isrc'].str.replace("-","")
meta = meta.drop_duplicates(subset = 'isrc', ignore_index = True)

In [5]:
meta.duplicated(subset = 'isrc').sum()

0

In [6]:
meta = meta.dropna(subset=['isrc']).reset_index(drop=True)
meta['isrc_year'] = meta['isrc'].str[5:7]
meta = meta.astype({'isrc_year':'int'})

In [7]:
meta['isrc_year'].unique()

array([21, 18, 20, 22, 19, 99,  1,  0, 17, 16, 14, 12,  8,  5, 15,  9, 82,
       30, 33, 77, 36, 35, 50, 62,  4, 70, 66,  6, 10, 96, 95, 23,  7, 11,
        3, 13,  2, 24, 81, 73, 60, 71, 65, 38, 85, 98, 63, 90, 94, 87, 80,
       78, 84, 88, 76, 74, 79, 92, 89, 91, 86, 97, 83, 51, 93, 25, 42, 26,
       69, 68, 67, 75, 58, 54, 57, 44, 64])

In [8]:
for i in range(len(meta.index)):
    if meta.loc[i,'isrc_year']>23:
        meta.loc[i,'isrc_year'] = meta.loc[i,'isrc_year']+1900
    else:
        meta.loc[i,'isrc_year'] = meta.loc[i,'isrc_year']+2000

In [9]:
meta['isrc_year'].unique()

array([2021, 2018, 2020, 2022, 2019, 1999, 2001, 2000, 2017, 2016, 2014,
       2012, 2008, 2005, 2015, 2009, 1982, 1930, 1933, 1977, 1936, 1935,
       1950, 1962, 2004, 1970, 1966, 2006, 2010, 1996, 1995, 2023, 2007,
       2011, 2003, 2013, 2002, 1924, 1981, 1973, 1960, 1971, 1965, 1938,
       1985, 1998, 1963, 1990, 1994, 1987, 1980, 1978, 1984, 1988, 1976,
       1974, 1979, 1992, 1989, 1991, 1986, 1997, 1983, 1951, 1993, 1925,
       1942, 1926, 1969, 1968, 1967, 1975, 1958, 1954, 1957, 1944, 1964])

In [10]:
meta.sample(10)

,track_id,dttm,title,language,isrc,genres,duration,isrc_year
25440,2b8b629ef24d703e4b252bc654e65ae5,2021-07-29 21:00:00,Afterglow,EN,DEYT72100539,[DANCE],121670.0,2021
55414,2436e2e05ffd4f9e6601d9d71add64f3,2020-09-14 19:03:10,Nicki Nicole Bzrp Session,None,QZMZ92079162,[POP],138380.0,2020
50621,c007126032b17292755c24bd1bc43aee,2016-06-02 21:24:56,Denk das ist OK,DE,DEM481200128,[ELECTRONICS],162450.0,2012
46223,c2223a990020d9dae1526d3e12353f3f,2014-02-03 20:00:00,Wild World,EN,TWA471109009,[POP],225050.0,2011
587,c97047b6ee89f2ed7a0bc3a5ba1e4088,2021-12-20 21:00:00,"Munchy Monk Circus (From ""Rhythm Heaven"")",None,AUXN22213034,[ELECTRONICS],85040.0,2022
54576,b91fb256d719297f8405d89c2fa4b68a,2020-08-17 16:23:27,Take Me Home,None,QZHZ62064781,"[POP, ASIAPOP]",156430.0,2020
2402,a9f589553c1af01132c9ab05102a7d96,2021-12-27 06:23:27,Pulang,EN,QZTAU2101857,[POP],303290.0,2021
39990,3ec2244bc9255f36353246f08f536329,2023-02-02 21:00:00,After Dark,EN,DGA082248664,[DANCE],139670.0,2022
27570,bbece65916eb5576b7609b33f3b8a972,2021-07-14 16:51:47,Cómo Fue,None,QZHZ62197341,"[FOLK, LATINFOLK]",207510.0,2021
66591,7283b1f443aef8e034a714ad776b6340,2018-10-07 21:00:00,Smooth Criminal,None,AEA0D1866274,[RAP],158090.0,2018


In [11]:
#клонируем репозиторий на диск
#Repo.clone_from('https://github.com/andreamust/MusicMetaLinker.git', 
#                to_path = 'D:/programs/projects/yandex_music/MusicMetaLinker')
#меняем директорию
os.chdir('D:/programs/projects/yandex_music/MusicMetaLinker')
#устанавливаем
!pip install -r requirements.txt

In [12]:
import MusicMetaLinker
from MusicMetaLinker.linking import linking

In [13]:
from MusicMetaLinker.linking import linking

# Create a linking object
linker = linking.Align(
#                artist="The Beatles",
#                album="Let It Be",
#                track="Let It Be",
#                track_number=4,
#                duration=168.25,
#                release_year=1970,
#                musicbrainz_id="b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d",
                isrc="USAR10000048",
                strict=False,
            )
# Link the metadata
track_name = linker.get_track()
artist_name = linker.get_artist()
album_name = linker.get_album()
track_number = linker.get_track_number()
duration = linker.get_duration()
release_year = linker.get_release_date()
isrc = linker.get_isrc()

# Get the identifiers
links = {#'musicbrainz': linker.get_mbid(),
         'isrc': linker.get_isrc() if isrc is None else isrc,
         #'deezer': linker.get_deezer_id(),
         #'deezer_url': linker.get_deezer_link(),
         #'youtube_url': linker.get_youtube_link(),
         }

Searching for None - None on YouTube Music


In [14]:
print(track_name)
print(release_year)
print(album_name)

Screaming in the Night
1993
Ugly is Beautiful: Shorter, Thicker & Uglier (Deluxe)


In [15]:
display(meta[7980:7990])
display(meta.sample(10))

,track_id,dttm,title,language,isrc,genres,duration,isrc_year
7980,50c6e97871a90ad5852aa03dc132f4f7,2022-03-18 17:29:16,Tarde Demais,None,QZHZ32226162,"[FOLK, LATINFOLK]",237660.0,2022
7981,78eca2af6f78bd29f286b75bc6ad700b,2022-03-18 17:29:16,Quem Ama Cuida,None,QZHZ32226163,"[FOLK, LATINFOLK]",244970.0,2022
7982,f0650ce0986c8eb2cf83376512039a7b,2022-03-18 17:29:48,Tenho Medo,None,QZHZ32226957,"[FOLK, LATINFOLK]",163290.0,2022
7983,1a8b336045d32b883c654ad0718f41b7,2022-03-18 17:29:48,Cabaré,None,QZHZ32226958,"[FOLK, LATINFOLK]",130440.0,2022
7984,617325aab678578417da7655437e586c,2022-03-18 17:29:48,Acaso,None,QZHZ32226959,"[FOLK, LATINFOLK]",225530.0,2022
7985,b78e3543e27c7ec994994dee71fc6f74,2022-03-18 17:29:48,Nega,None,QZHZ32226960,"[FOLK, LATINFOLK]",182360.0,2022
7986,16614150adfcaf7695b441f0c176ff02,2022-03-18 17:29:48,Confidencial,None,QZHZ32226961,"[FOLK, LATINFOLK]",151030.0,2022
7987,5c3bead4de2ad70ef448a8937058c680,2022-03-18 17:29:48,Rita,None,QZHZ32226962,"[FOLK, LATINFOLK]",130420.0,2022
7988,2b3c755b2486e657d9a805e612ce5b42,2022-03-18 17:29:48,Amor de Rapariga,None,QZHZ32226963,"[FOLK, LATINFOLK]",192000.0,2022
7989,d767b6b1f147cdcdda8a48bc48ac5854,2022-03-18 17:29:48,Nem por um Milhão,None,QZHZ32226964,"[FOLK, LATINFOLK]",178200.0,2022


,track_id,dttm,title,language,isrc,genres,duration,isrc_year
37957,4304819d69a0086646dfd77c9d852fc6,2021-10-26 18:43:34,Pininfarina,ES,GBKPL2170332,"[FOLK, LATINFOLK]",173560.0,2021
15557,1fda690cf9c3a93ca4d07653e49cb250,2019-07-02 21:00:00,"Любовь, похожая на сон",None,TCAEH1933774,"[POP, RUSPOP]",261200.0,2019
64176,f6a29132f8571fa320fa4bfdee21b63a,2021-03-25 03:31:33,"Frappe Snowland & Sherbert Land (From ""Mario K...",None,AUXN22143399,[SOUNDTRACK],129000.0,2021
27397,726109c8cf1e83cdba829d751b925e71,2021-07-12 19:12:41,Mi Buen Amor,None,QZHZ62177439,"[ROCK, ALLROCK]",218630.0,2021
54091,bf26a551c1575864b9f5f5934a3081a8,2020-07-27 00:21:55,Circles,None,QZHZ52062907,[POP],181130.0,2020
36698,cdac5505fb117d6c9f6b2c2d47ff4c1d,2021-10-14 01:53:59,Dueles Tan Bien,None,QZNJX2151733,[ALTERNATIVE],187830.0,2021
28694,dc6d668816c375eef78a523220dbcb15,2021-07-27 23:22:00,Meu Cenário,None,QZMZ92148186,"[FOLK, LATINFOLK]",105280.0,2021
36743,8cd2ea8a2336c231120266b4163df13c,2021-10-14 18:00:00,Edinalva,None,QZNJX2138265,"[FOLK, LATINFOLK]",173980.0,2021
52959,c86a85420adbb968abe7de9b51793b26,2018-01-09 10:56:29,Группа крови,None,RUB421401399,"[RUSROCK, ALLROCK]",239820.0,2014
3571,a61a32f14dc8266f82628edc99a85fe0,2022-01-18 21:00:00,Oh My God,EN,QZN882203281,[ELECTRONICS],272830.0,2022


In [ ]:
for i in range(7900, 30000):
    linker = linking.Align(
                track=meta.loc[i,'title'],
                isrc=meta.loc[i,'isrc'],
                strict=False,
            )
    # Link the metadata
    #track_name = linker.get_track()
    #artist_name = linker.get_artist()
    #album_name = linker.get_album()
    #track_number = linker.get_track_number()
    #duration = linker.get_duration()
    release_year = linker.get_release_date()
    isrc = linker.get_isrc()
    meta.loc[i, 'linker_release_year'] = linker.get_release_date()
    meta.loc[i, 'linker_artist_name'] = linker.get_artist()
    meta.loc[i, 'linker_track_name'] = linker.get_track()
    meta.loc[i, 'linker_album_name'] =  linker.get_album()

    # Get the identifiers
    links = {'musicbrainz': linker.get_mbid(),
             'isrc': linker.get_isrc() if isrc is None else isrc,
             'deezer': linker.get_deezer_id(),
             'deezer_url': linker.get_deezer_link(),
             'youtube_url': linker.get_youtube_link(),
             }
display(meta[7980:8000])
display(meta.sample(20))


Searching for None - Un Buen Día para Vos on YouTube Music


C:\Temp\ipykernel_9856\4146494790.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2022' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  meta.loc[i, 'linker_release_year'] = linker.get_release_date()
C:\Temp\ipykernel_9856\4146494790.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Dire Straits' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  meta.loc[i, 'linker_artist_name'] = linker.get_artist()
C:\Temp\ipykernel_9856\4146494790.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Un Buen Día para Vos' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  meta.loc[i, 'linker_track_name'] = linker.get_track()
C:\Temp\ipykernel_9856\4146494790.p

Searching for None - A Plena Luz on YouTube Music
Searching for None - Fuiste Mala on YouTube Music
Searching for None - Lucha de Gigantes on YouTube Music
Searching for None - Solo Pienso en Ti on YouTube Music
Searching for None - Me Llamas on YouTube Music
Searching for None - Me Cuesta Tanto Olvidarte on YouTube Music
Searching for None - En Realidad on YouTube Music
Searching for None - En Algun Lugar on YouTube Music
Searching for None - Procuro Olvidarte on YouTube Music
Searching for None - El Tocayito on YouTube Music
Searching for None - Patito Patito on YouTube Music
Searching for None - Polka Elegancia on YouTube Music
Searching for None - El Texanito on YouTube Music
Searching for None - Polka Stocky on YouTube Music
Searching for None - De China a Bravo on YouTube Music
Searching for None - Bam Bam on YouTube Music
Searching for None - Como Si No Importara on YouTube Music
Searching for None - Hazmelo on YouTube Music
Searching for None - Mi Debilidad on YouTube Music
Sea

In [ ]:
linker.get_release_date()

In [ ]:
meta.to_csv('D:\programs\projects\yandex_music\meta_enriched2.csv')

In [ ]:
print(meta[meta['title'] =='Surface Tension 4'])